# Running multiple calculations on a given model


## Aim

This notebook shows how we can run multiple calculations of a given structure

### Setup

The initial setup is very similar to the other tutorials, such as `singlepoint.ipynb`, which goes into more detail about what each step is doing

Load the aiida profile and code:

In [ ]:
from aiida import load_profile
load_profile()

In [ ]:
from aiida_mlip.data.model import ModelData
uri = "https://github.com/stfc/janus-core/raw/main/tests/models/mace_mp_small.model"
model = ModelData.from_uri(uri, architecture="mace_mp", cache_dir="mlips")

In [ ]:
from aiida.orm import load_code
janus_code = load_code("janus@localhost")
qe_code = load_code("pw@localhost")

Inputs should include the model, code, metadata, and any other keyword arguments expected by the calculation we are running:

In [ ]:
from aiida.orm import Str, Float, Bool
inputs = {
    "code": janus_code,
    "model": model,
    "arch": Str(model.architecture),
    "precision": Str("float64"),
    "device": Str("cpu"),
    "fmax": Float(0.1), 
    "opt_cell_lengths": Bool(False), 
    "opt_cell_fully": Bool(True), 
    "metadata": {"options": {"resources": {"num_machines": 1}}},
    }

We must now choose the calculations to perform:

In [ ]:
from aiida.plugins import CalculationFactory
geomoptCalc = CalculationFactory("mlip.opt")
descriptorsCalc = CalculationFactory("mlip.descriptors")


Now we can create our WorkGraph. This includes passing in the inputs, checking the amount of structures we have and interating through. Note for this workbook we have decreased the amount of structures to two for the sake of simplicity. In the loop we can call each structure; run geomopt calculation, pass the output file into descriptors calculation, and get the final outputs of all the structures

In [ ]:
from sample_split import build_filters_workgraph
from aiida.orm import Str, Float, Bool, Int
from ase.io import read

structure = "../../examples/tutorials/structures/lj-traj.xyz"

split_inputs = {
    "config_types": Str(""),
    "prefix": Str(""),
    "scale": Float(1.0e5),
    "append_mode": Bool(False)
}

wg = build_filters_workgraph(
    initial_struct = structure,
    calc_inputs = [geomoptCalc,descriptorsCalc],
    wg_inputs= inputs,
    split_inputs = split_inputs
)



In [ ]:
wg


In [ ]:
wg.run()

In [ ]:
wg.tasks.process_and_split_data.outputs.result.value.get_dict()

Now we can run the calculations

In [9]:
from aiida.orm import load_node
from aiida import load_profile


load_profile("presto", allow_switch=True)
file_dict = load_node(3588).get_dict()

In [19]:
from ase.io import read

ase=read(file_dict['train_file'], index=7)

print(ase)

Atoms(symbols='Ar100', pbc=False, cell=[17.395296987042, 17.395296987042, 17.395296987042], masses=...)


In [ ]:
from ase.build import bulk
from aiida.orm import Dict, KpointsData, StructureData, load_code, load_group
from aiida.plugins.factories import WorkflowFactory
from aiida import load_profile
from ase.io import read
from aiida_workgraph import WorkGraph
from aiida_quantumespresso.calculations.pw import PwCalculation

qe_code = load_code("pw@localhost")
kpoints = KpointsData()
kpoints.set_kpoints_mesh([1, 1, 1])
pseudo_family = load_group('SSSP/1.3/PBE/efficiency')
metadata =  {
    "options": {
        "resources": {
            "num_machines": 1
        }
    }
}


for _, filepath_out in file_dict:
    for i in range(len(ase=read(filepath_out, index=':'))):
        structure = StructureData(ase=read(filepath_out, index=i))
        pseudos = pseudo_family.get_pseudos(structure=structure)
        ecutwfc, ecutrho = pseudo_family.get_recommended_cutoffs(
            structure=structure,
            unit='Ry',
        )
        pw_paras = Dict(
            {
                "CONTROL": {
                    "calculation": "scf",
                },
                "SYSTEM": {
                    "ecutwfc": ecutwfc,
                    "ecutrho": ecutrho,
                },
            }
        )


        wg = WorkGraph("QE")

        inputs = {
            "code": qe_code,
            "parameters": pw_paras,
            "kpoints": kpoints,
            "pseudos": pseudos,
            "metadata": metadata,
            "structure": structure,
        }
        qe_task = wg.add_task(
            PwCalculation,
            **inputs
        )


In [ ]:
wg

In [ ]:
wg.run()

# TEST

In [ ]:
from aiida.orm import load_node
from aiida import load_profile

load_profile()

sfdata = load_node(pk= 15657)
sfdata1 = load_node(pk= 15661)

structs_out = {
    "structs0" : sfdata,
    "structs1" : sfdata1,
}